# Сбор данных для проекта по АОЕЯ "Корпус анекдотов"

Парсинг

Парсинг ВК сообщества с анекдотами. Собираем анекдоты и мета-информацию о них

In [ ]:
import requests
import pandas as pd

Запишем токен (*здесь скрыт*) и версию.

In [ ]:
VERSION = "5.130"

In [ ]:
wall_get_url = "https://api.vk.com/method/wall.get"

In [ ]:
offs = [x*100 for x in range(0, 220)]  # генерируем оффсеты

In [ ]:
all_the_data = []
for off in offs:
    data = requests.get(
        wall_get_url, 
        params={
            "owner_id": -50384057,  # id группы, номер пишется после "-"
            "count": 100,
            "v": VERSION,
            "access_token": TOKEN,
            "offset": off, 
        }
    ).json()
    all_the_data.append(data)

In [ ]:
texts_data = []

Вытаскиваем тексты из постов

In [ ]:
i = 0
for data in all_the_data:
    texts = []
    if len(data['response']['items']) != 0:
        text = data['response']['items'][0]['text']
        text_tokens = data['response']['items'][0]['text'].split()
        if len(text_tokens) > 99: #в тексте должно быть как минимум 100 токенов
            texts.append(i)
            texts.append(data['response']['items'][0]['id'])
            texts.append(f"https://vk.com/wall{data['response']['items'][0]['from_id']}_{data['response']['items'][0]['id']}")
            texts.append(text)
            texts.append(text_tokens)
            text_sntnss = text.split('.')
            texts.append(text_sntnss)
            i += 1
    texts_data.append(texts)

In [ ]:
texts_data2 = [text_data for text_data in texts_data if text_data]

Делаем датафрейм с помощью ``pandas``

In [ ]:
df = pd.DataFrame(texts_data2, columns = ['id', 'vk_id', 'link', 'text', 'tokens', 'sentences'])

df.head()

Записываем в два формата: ``.xlsx`` и ``.csv``

In [ ]:
writer = pd.ExcelWriter('vk_texts.xlsx', engine='xlsxwriter')
df.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
df.to_csv('csv_vk_texts.csv')

## Морфология

Парсим скачанные тексты с помощью NLP

In [ ]:
!pip install stanza
import stanza
stanza.download('ru')

С текстами были некоторые проблемы, которые мы решили препроцессингом: тексты начинались с латинских симоволов, которые были заменены на кириллические:

In [ ]:
import csv
import nltk
from nltk.tokenize import sent_tokenize
import re
rows = ''
nltk.download('punkt')
from string import punctuation
my_parse = {}
with open ('csv_vk_texts.csv') as f:
  reader = csv.reader(f, delimiter = ',')
  for row in reader:
    rows += row[4] + '#__#'
  alph = {'a':'а','b':'в', 'c':'с', 'd':'о', 'e':'е', 'h':'н', 'k':'к', 'm':'м', 'o':'о', 'p':'р', 't':'т', 'u' :'и', 'x':'х', 'y':'у', 'A':'А', 'B':'В', 'C':'С', 'E':'Е', 'H':'Н', 'K':'К', 'M':'М', 'N':'Н', 'O':'О', 'P':'Р', 'T':'Т', 'X':'Х', 'Y':'У'}
  for key in alph:
    rows = rows.replace(key, alph[key]) 
  rows_new = rows.split('#__#')
  rows_new.remove(rows_new[0])
  for idx, row in enumerate(rows_new):
    text_sents = sent_tokenize(row)  # делим на предложения, чтобы знать их индексы в тексте
    #words_new = re.sub('T', 'т', row[3])
    #print(words)
    sent_idx = {}
    count = 0
    for sent in text_sents:
      sent_idx[count] = sent
      count += 1
    my_parse[idx] = sent_idx
print(my_parse)

In [ ]:
nlp = stanza.Pipeline(lang='ru', processors='tokenize,lemma,pos')
def stanza_parse(sent):
  doc = nlp(sent)
  new_list = []
  for sent in doc.sentences:
    for word in sent.words:
      if word.upos != 'PUNCT':
        new_list.append((word.text, word.lemma, word.upos))
  return new_list
#print(stanza_parse(words))

In [ ]:
parse_dict = {}

for text_id, sents in my_parse.items():
  text_dict = {}
  for idx, elem in sents.items():
    text_dict[idx] = stanza_parse(elem)
  parse_dict[text_id] = text_dict

Пишем файлы: первый для предложений и их индексов. а другой для словоформ и тегов

In [ ]:
with open('sents_id.csv', 'w') as sents_file:
    writer = csv.writer(sents_file)
    writer.writerow(['id_text', 'id_sent', 'sent'])

    for key, value in my_parse.items():
        id_text = key
        for x, y in value.items():
            id_sent = x
            sent = y
            writer.writerow([id_text, id_sent, sent])

In [ ]:
with open ('word_pos_lemma_stanza.csv', 'w') as file:
  writer = csv.writer(file)
  writer.writerow(['id_text', 'id_sent','word', 'lemma', 'pos'])

  for key, value in parse_dict.items():
    id_text = key
    #print(value)
    for x, y in value.items():
      id_sent = x
      for listt in y:
        word = listt[0].lower()
        lemma = listt[1].lower()
        pos = listt[2]
        writer.writerow([id_text, id_sent, word, lemma, pos])

## База данных

Создаём базу данных и складываем туда обработанные данные

In [10]:
import sqlite3
import csv

In [11]:
con = sqlite3.connect('anecdotes.db')
cur = con.cursor()

In [12]:
cur.execute('''
CREATE TABLE IF NOT EXISTS texts_meta
(id INTEGER PRIMARY KEY,
vk_id text,
text text,
link text)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS morph_parse
(id INTEGER PRIMARY KEY AUTOINCREMENT,
text_id int,
sent_id int,
token text,
lemma text,
POS text)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS sents
(id INTEGER PRIMARY KEY AUTOINCREMENT,
text_id int,
sent_id int,
sent text)
''')

con.commit()

Записываем данные в базу:

In [13]:
with open ('csv_vk_texts.csv') as texts_csv:
    texts_reader = csv.DictReader(texts_csv, delimiter=',')
    
    for text_row in texts_reader:
        text_id = text_row['id']
        vk_id = text_row['vk_id']
        link = text_row['link']
        text = text_row['text']

        cur.execute('''INSERT INTO texts_meta (id, vk_id, text, link)
        VALUES (?, ?, ?, ?)''',
        (int(text_id), vk_id, text, link))

In [16]:
with open ('word_pos_lemma_stanza.csv') as morph_csv:
    morph_reader = csv.DictReader(morph_csv, delimiter=',')

    for morph_row in morph_reader:
        text_id = morph_row['id_text']
        sent_id = morph_row['id_sent']
        token = morph_row['word']
        lemma = morph_row['lemma']
        pos = morph_row['pos']

        cur.execute('''INSERT INTO morph_parse (text_id, sent_id, token, lemma, POS)
        VALUES (?, ?, ?, ?, ?)''',
        (int(text_id), int(sent_id), token, lemma, pos))

In [19]:
with open ('sents_id.csv') as sents_csv:
    sents_reader = csv.DictReader(sents_csv, delimiter=',')

    for sent_row in sents_reader:
        sent_id = sent_row['id_sent']
        text_id = sent_row['id_text']
        sent = sent_row['sent']

        cur.execute('''INSERT INTO sents (text_id, sent_id, sent)
        VALUES (?, ?, ?)''',
        (int(text_id), int(sent_id), sent))

In [20]:
con.commit()
con.close()